In [ ]:
import pandas as pd

def load_merge_df():
    splits = ["1_train", "1_test", "2"]

    specs = [
        # (model_display_name, base_path_with_{split})
        ("o4-mini (best reasoning)", "../results/qa/qa_strat_6_split_{}.csv"),
        ("Mistral (worst)",     "../results/qa_vllm/qa_strat_4_split_{}_vllm.csv"),
        ("Gemma (best open)",         "../results/qa_vllm2/qa_strat_4_split_{}_vllm.csv"),
    ]

    frames = []

    for model_name, base in specs:
        for split in splits:
            base_path = base.format(split)
            df = pd.read_csv(base_path)

            sub = df[["id", "output_prompt", "exact_match", "input_prompt"]].copy()
            sub.columns = ["id", "output_prompt", "exact_match", "input_prompt"]
            sub.insert(0, "model", model_name)
            sub.insert(1, "split", split)

            frames.append(sub)

    agg = pd.concat(frames, ignore_index=True)
    return agg

import pandas as pd
import ast

def load_inputs():
    df1 = load_merge_df()
    qa_train = pd.read_csv("../data/dataset_V3/split_1_train.csv", index_col=False)
    qa_test  = pd.read_csv("../data/dataset_V3/split_1_test.csv", index_col=False)
    qa_test2  = pd.read_csv("../data/dataset_V3/split_2.csv", index_col=False)
    qa_refs = pd.concat([qa_train, qa_test, qa_test2], ignore_index=True)
    qa_refs = qa_refs[["id", "question", "answers" ,"explanation"]]
    merged_df = pd.merge(df1, qa_refs, on=["id"], how="inner")
    return merged_df

In [ ]:
llm_judge_prompt_2 = """Vei primi:
- O întrebare grilă (răspuns multiplu) din testele auto în limba română.
- Documentele legale folosite de o persoană pentru a răspunde.
- Explicația și răspunsul final oferit de acea persoană.
- Explicația oficială și răspunsul corect.

Scopul tău este să identifici și să marchezi următoarele tipuri de erori (True/False), comparând <RaspunsOferit> cu <ExplicatieOficiala> și <DocumenteReferinta>. Nu inventa explicații noi și nu analiza concepte generale care nu apar în documente. Marchează <True> doar dacă există dovezi clare în text.

Categoriile de erori:

1. [Supra-interpretare]  
Persoana introduce în raționament reguli, condiții sau consecințe care nu sunt prevăzute explicit în documentele legale.  
Apare atunci când candidatul extrapolează dintr-o regulă reală pentru a susține o concluzie fără bază legală: extinde aplicabilitatea unei sancțiuni, inventează o condiție suplimentară sau tratează o formulare generală ca interdicție absolută.  
**Tipare frecvente:** combinarea nejustificată a mai multor articole, adăugarea de distanțe/valori inexistente, presupunerea unor sancțiuni automate.  
**Exemple:** a considerat că prezența unui polițist implică automat obligația de a opri; a extrapolat principiul general al siguranței la o interdicție neprevăzută; a impus o viteză minimă neprevăzută în text.  

2. [Neglijare]  
Persoana nu ia în considerare informații relevante sau excepții cruciale din documente ori din.  
A consultat textele corecte dar a ignorat condiții esențiale (excepții, restricții, situații speciale).  
**Tipare frecvente:** omiterea excepțiilor legale (tramvaie, situații speciale), ignorarea condițiilor cumulative, omisiunea diferențierii între categorii de vehicule sau drumuri.  
**Exemple:** a ignorat excepția care permite depășirea tramvaielor fără refugiu; a neglijat condițiile pentru priorități în zone rezidențiale; a citit corect un articol dar a omis altul complementar.

3. [Interpretare eronată]  
Persoana citește sau aplică greșit prevederile legale existente, fără a adăuga reguli noi sau a confunda concepte.  
Greșeala apare din interpretarea incorectă a textului: aplicare literală fără context, ignorarea priorității între reguli, citire selectivă.  
**Tipare frecvente:** aplicarea marcajului continuu ca interdicție absolută, neînțelegerea priorității între semnal și indicator, interpretarea eronată a duratelor de suspendare.  
**Exemple:** a considerat că marcajul continuu interzice orice depășire; a ignorat faptul că indicatorul „înainte sau la dreapta” rămâne valabil la semnal verde; a aplicat sancțiuni din altă categorie.

4. [Valori]  
Persoana aplică greșit o valoare numerică, un prag legal sau un standard tehnic prevăzut în textele normative.  
Eroarea nu este doar de lectură, ci de aplicare incorectă a pragurilor sau standardelor (viteze, distanțe, mase, procente, puncte etc.) sau folosirea valorii corecte în regimul juridic greșit.  
**Tipare frecvente:** confundarea limitelor de viteză între drumuri expres/E, aplicarea unei distanțe greșite (25m vs 50m), folosirea valorilor valabile pentru alte categorii de vehicule, inversarea punctelor sau aplicarea valorilor corecte în context greșit.  
**Exemple:** a considerat că limita pe drumurile expres este 100 km/h (valoare pentru drumuri E); a tratat „la mai puțin de 25 m” ca „la mai puțin de 50 m”; a atribuit 9 puncte în loc de 6; a aplicat masa maximă admisă din categoria N la vehicule B.  
**Diferență față de „Lipsă de atenție la detalii”**: la Valori, persoana aplică activ o valoare greșită; la Lipsă de atenție, valoarea e corectă în document dar a fost ignorată sau citită superficial.

5. [Recomandări]  
Persoana confundă caracterul juridic al unei prevederi: tratează o recomandare ca pe o obligație legală strictă sau, invers, minimalizează o recomandare relevantă pentru răspunsul corect.  
Eroarea apare prin neînțelegerea expresiilor „de regulă”, „se recomandă”, „în mod normal”, care nu au același statut juridic ca obligațiile imperative.  
**Tipare frecvente:** interpretarea „de regulă” ca regulă absolută; eliminarea variantelor formulate ca recomandări; ignorarea recomandărilor relevante pentru răspuns; tratarea recomandărilor ca neimportante în fața obligațiilor.  
**Exemple:** a tratat recomandarea de a folosi luminile ziua pe drumurile naționale ca obligație legală strictă; a ignorat recomandarea de semnalizare în lipsa altor vehicule; a considerat „de regulă” ca o normă fără excepții.

Instrucțiuni (versiune îmbunătățită):
- Obiectiv: explică *de ce* <RaspunsOferit> este greșit (mecanismul erorii) raportat la <DocumenteReferinta>, NU „față de <ExplicatieOficiala>”.

- Ordinea de lucru:
  1) Extrage regulile/condițiile relevante din <DocumenteReferinta>.
  2) Extrage afirmațiile-cheie din <RaspunsOferit>.
  3) Compară (2) cu (1) și identifică mecanismul erorii (categoria potrivită).
  4) Folosește <ExplicatieOficiala> DOAR pentru orientare (ca să înțelegi care e concluzia corectă și ce articole sunt relevante), NU ca probă în explicație.

- Evidență necesară pentru <True>:
  Marchează <True> la o categorie numai dacă poți formula o frază scurtă în care:
  a) indici concret ce susține <RaspunsOferit>, și
  b) arăți scurt ce prevăd textele din <DocumenteReferinta> (număr articol/condiție/valoare/direcție) care contrazic, restrâng sau nuanțează acea susținere.
  Dacă nu există astfel de dovezi în <DocumenteReferinta>, marchează <False> chiar dacă <ExplicatieOficiala> indică altceva.

- Interdicții explicite:
  • NU cita, NU rezuma și NU invoca <ExplicatieOficiala> în <explicatie>.
  • NU scrie „conform/după/în explicația oficială…”, „nu a urmat <ExplicatieOficiala>”, „răspunsul corect este X în <ExplicatieOficiala>”.
  • NU inventa reguli, articole sau condiții care nu apar în <DocumenteReferinta>.

- Stilul explicațiilor:
  • 1–2 fraze per criteriu, neutre și factuale, fără meta-comentarii.
  • Folosește, când e posibil, referințe minimale la articol/alin./lit. din <DocumenteReferinta> (ex.: „art. 120 lit. b) interzice…”, „Regulament-123 h) condiționează de…”).
  • Evită formulări vagi de tipul „textul sugerează”, „pare că”, „probabil”.

- Când pui <False>:
  • Documentele nu acoperă situația sau nu contrazic clar <RaspunsOferit>.
  • Diferența e vizibilă doar în <ExplicatieOficiala>, nu și în <DocumenteReferinta>.
  • Există ambiguitate rezonabilă între text și răspuns.

- Format de ieșire:
  • Răspunde STRICT în XML, o singură secțiune <verdict>, fără text suplimentar.
  • NU genera niciodată JSON.
  • Pentru fiecare <criteriu>: <categorie>, <explicatie> (1–2 fraze bazate pe <DocumenteReferinta>), <valoare>True/False</valoare> (fix aceste forme).

- NU FOLOSI NICIODATA: "conform explicatiei oficiale"
  
- Exemple rapide (doar ca ghid intern; NU le include în output):
  ✔ Acceptat: „A tratat interdicția ca absolută, deși art. 120 prevede excepția …; eroare de Generalizare.”
  ✘ Respins: „Persoana nu a luat în considerare că răspunsul corect este C, conform explicației oficiale.”, "Persoana a omis să observe că explicația oficială menționează explicit"
  Exemplele respinse arata faptul ca nu trebuie sa compari in mod direct raspunsul si explicatia oficiala. Persoana nu a avut acces la explicatia oficiala cand a dat raspunsul.

Iată detaliile de analizat:

<Intrebare>
{question}
</Intrebare>

<VarianteRaspuns>
{options_refs}
</VarianteRaspuns>

<VarianteCorecte>
{options_correct}
</VarianteCorecte>

<DocumenteReferinta>
{documents_ref}
</DocumenteReferinta>

<RaspunsOferit>
{answer}
</RaspunsOferit>

<RaspunsCorect>
{is_correct}
</RaspunsCorect>

<ExplicatieOficiala>
{answer_ref}
</ExplicatieOficiala>

Returnează rezultatul în structura:

<verdict>
  <criteriu id="1">
    <categorie>Supra-interpretare</categorie>
    <explicatie>...</explicatie>
    <valoare>True/False</valoare>
  </criteriu>
  <criteriu id="2">
    <categorie>Neglijare</categorie>
    <explicatie>...</explicatie>
    <valoare>True/False</valoare>
  </criteriu>
  <criteriu id="3">
    <categorie>Interpretare eronată</categorie>
    <explicatie>...</explicatie>
    <valoare>True/False</valoare>
  </criteriu>
  <criteriu id="4">
    <categorie>Valori</categorie>
    <explicatie>...</explicatie>
    <valoare>True/False</valoare>
  </criteriu>
  <criteriu id="5">
    <categorie>Recomandări</categorie>
    <explicatie>...</explicatie>
    <valoare>True/False</valoare>
  </criteriu>
</verdict>

===========
"""




In [11]:
def judge(item):
    # print(item)
    answers = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
    correct_answers = [entry['answer_text'][0] for entry in ast.literal_eval(item['answers']) if entry['is_correct']]

    documents = item["input_prompt"]
    documents = documents.split("Aceastea sunt legile relevante, dar nu neaparat toate sunt relevante:")[1]
    documents = documents.split("===================")[0]
    input_prompt = llm_judge_prompt_2.format(
        question=item["question"],
        options_refs=answers,
        options_correct=correct_answers,
        documents_ref=documents,
        answer=item["output_prompt"],
        is_correct="Raspunsul final este corect" if item["exact_match"] else "Raspunsul final este incorect",
        answer_ref=item["explanation"]
    )
    return input_prompt

In [12]:
import tqdm
import os
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

os.environ["OPENAI_BASE_URL"] = "http://localhost:8889/v1"
os.environ["OPENAI_API_KEY"] = "x"

SAVE_PATH = "./llm_judge_out_2/gemma-3-27b-it.csv"

def call_llm(sys_prompt):
    try:
        response = openai.chat.completions.create(
            model="google/gemma-3-27b-it",
            messages=[
                {"role": "user", "content": sys_prompt},
            ],
            temperature=0,
            max_tokens=2000,
            seed=25
            # seed=46 
        )

        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling LLM: {e}")
        return None

# def mass_qa_runner(data: pd.DataFrame, save_path: str = SAVE_PATH):
#     # Load existing results if they exist
#     if os.path.exists(save_path):
#         results_df = pd.read_csv(save_path)
#     else:
#         results_df = pd.DataFrame(columns=["model", "split", "id", "result"])

#     # Use tuple (model, split, id) as unique identifier
#     processed_keys = set(
#         tuple(row) for row in results_df[["model", "split", "id"]].to_numpy()
#     )

#     for real_idx, (idx, item) in tqdm.tqdm(enumerate(data.iterrows()), total=len(data)):
#         key = (item["model"], item["split"], item["id"])

#         if key in processed_keys:
#             continue  # Skip already processed

#         prompt = judge(item)  # assumes you have a judge() function
#         result = call_llm(prompt)

#         # Append to results if successful
#         if result is not None:
#             new_row = {
#                 "model": item["model"],
#                 "split": item["split"],
#                 "id": item["id"],
#                 "result": result,
#             }
#             results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

#             # Save intermediate progress
#             results_df.to_csv(save_path, index=False)

#             # Update processed set
#             processed_keys.add(key)

#     return results_df


def process_row(idx, item, processed_keys):
    key = (item["model"], item["split"], item["id"])
    if key in processed_keys:
        return None

    prompt = judge(item)
    result = call_llm(prompt)

    if result is None:
        return None

    return {
        "model": item["model"],
        "split": item["split"],
        "id": item["id"],
        "result": result,
    }

def run_parallel(data, save_path=SAVE_PATH, max_workers=64):
    # Load existing results if they exist
    if os.path.exists(save_path):
        results_df = pd.read_csv(save_path)
    else:
        results_df = pd.DataFrame(columns=["model", "split", "id", "result"])

    # Use tuple (model, split, id) as unique identifier
    processed_keys = set(
        tuple(row) for row in results_df[["model", "split", "id"]].to_numpy()
    )

    results_df = pd.DataFrame()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_row, idx, item, processed_keys): (idx, item)
            for idx, item in data.iterrows()
        }

        for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
            row = future.result()
            if row is not None:
                results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
                results_df.to_csv(save_path, index=False)  # save progress incrementally

    return results_df

# Example usage
merged_df = load_inputs()
merged_df = merged_df[merged_df["exact_match"] == False]
result_df = run_parallel(merged_df)
print(result_df.head())


100%|██████████| 794/794 [18:09<00:00,  1.37s/it]

                      model    split        id  \
0  o4-mini (best reasoning)  1_train  0641a4e4   
1  o4-mini (best reasoning)  1_train  376f1dc6   
2  o4-mini (best reasoning)  1_train  7f68bad6   
3  o4-mini (best reasoning)  1_train  e0f85457   
4  o4-mini (best reasoning)  1_train  ee8de379   

                                              result  
0  ```xml\n<verdict>\n  <criteriu id="1">\n    <c...  
1  ```xml\n<verdict>\n  <criteriu id="1">\n    <c...  
2  ```xml\n<verdict>\n  <criteriu id="1">\n    <c...  
3  ```xml\n<verdict>\n  <criteriu id="1">\n    <c...  
4  ```xml\n<verdict>\n  <criteriu id="1">\n    <c...  


# K Means clustering

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# --- Load Data ---
SAVE_PATH = "./llm_gemma_redhat_judge_7.csv"
df = pd.read_csv(SAVE_PATH)

# --- Split 'result' column by newlines into multiple rows ---
expanded_rows = []
for idx, row in df.iterrows():
    if pd.isna(row['result']):
        continue
    for sentence in str(row['result']).split('\n'):
        sentence = sentence.strip()
        if sentence:  # avoid empty lines
            expanded_rows.append({
                'model': row['model'],
                'split': row['split'],
                'id': row['id'],
                'result': sentence
            })

df_expanded = pd.DataFrame(expanded_rows)
print(f"Original rows: {len(df)}, Expanded rows: {len(df_expanded)}")
print(df_expanded.head())

# --- Load SentenceTransformer model ---
model = SentenceTransformer("intfloat/multilingual-e5-large")

# --- Embed the 'result' column ---
texts = df_expanded['result'].tolist()
embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)

# --- Run KMeans ---
num_clusters = 15
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df_expanded['cluster'] = kmeans.fit_predict(embeddings)

# --- Inspect cluster distribution ---
print("\nCluster counts:")
print(df_expanded['cluster'].value_counts())

# --- Print sentences grouped by cluster ---
pd.set_option('display.max_colwidth', None)

for c in sorted(df_expanded['cluster'].unique()):
    print(f"\n=== Cluster {c} ===")
    cluster_sentences = df_expanded[df_expanded['cluster'] == c]['result']
    for i, sentence in enumerate(cluster_sentences, 1):
        print(f"{i}. {sentence}")

# --- Optionally, save clustered results ---
df_expanded.to_csv("./llm_gemma_redhat_judge_7_clustered_by_line.csv", index=False)
print("\nClustered CSV saved to llm_gemma_redhat_judge_7_clustered_by_line.csv")
